In [ ]:
'''
This code scrapes all reviews for Airbnb listings using Selenium, provided a list of pre selected listings 
(in this case, NYC listings filtered for last review from 2017 till today). The original data does not include reviews.
'''

In [176]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_binary
import time
import pyautogui
import os
import sys
import pandas as pd
from datetime import datetime

In [199]:
data = pd.read_csv('/Users/Giuliana/Documents/GradSchool/Winter Term/BAX 422 - DDR/final_project/airbnb_data_files/NYC_listings_2021_last_review_2017.csv', 
                   na_values = "N/A")

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (67) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [200]:
# We have 25,913 airbnb listings originally within our universe, with 74 features
data.shape

(25913, 74)

In [196]:
data.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2595,https://www.airbnb.com/rooms/2595,2.020000e+13,12/5/21,Skylit Midtown Castle,"Beautiful, spacious skylit studio in the heart...",Centrally located in the heart of Manhattan ju...,https://a0.muscache.com/pictures/f0813a11-40b2...,2845,https://www.airbnb.com/users/show/2845,...,4.79,4.86,4.41,NaN,f,3,3,0,0,0.33
1,3831,https://www.airbnb.com/rooms/3831,2.020000e+13,12/5/21,"Whole flr w/private bdrm, bath & kitchen(pls r...","Enjoy 500 s.f. top floor in 1899 brownstone, w...",Just the right mix of urban center and local n...,https://a0.muscache.com/pictures/e49999c2-9fd5...,4869,https://www.airbnb.com/users/show/4869,...,4.80,4.71,4.64,NaN,f,1,1,0,0,4.86
2,5121,https://www.airbnb.com/rooms/5121,2.020000e+13,12/5/21,BlissArtsSpace!,<b>The space</b><br />HELLO EVERYONE AND THANK...,NaN,https://a0.muscache.com/pictures/2090980c-b68e...,7356,https://www.airbnb.com/users/show/7356,...,4.91,4.47,4.52,NaN,f,2,0,2,0,0.52
3,5136,https://www.airbnb.com/rooms/5136,2.020000e+13,12/5/21,"Spacious Brooklyn Duplex, Patio + Garden",We welcome you to stay in our lovely 2 br dupl...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,7378,https://www.airbnb.com/users/show/7378,...,5.00,4.50,5.00,NaN,f,1,1,0,0,0.02
4,5178,https://www.airbnb.com/rooms/5178,2.020000e+13,12/5/21,Large Furnished Room Near B'way,Please don’t expect the luxury here just a bas...,"Theater district, many restaurants around here.",https://a0.muscache.com/pictures/12065/f070997...,8967,https://www.airbnb.com/users/show/8967,...,4.42,4.87,4.36,NaN,f,1,0,1,0,3.68


In [219]:
# Start driver
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/Users/Giuliana/.wdm/drivers/chromedriver/mac64/99.0.4844.51/chromedriver] found in cache
<ipython-input-219-7b5228126294>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [220]:
# Create dataframe to store reviews data
all_reviews_df = pd.DataFrame(columns = ['listing_id', 'url', 'review_posted_date', 'review'])

# Loop through all URLs to scrape all reviews
for url, listing_id in zip(urls_list[0:2000], ids_list[0:2000]):
    
    # Open to url
    driver.get(url)
    time.sleep(3)
    
    ## This will catch if URL is still valid (ie, if listing is still up on Airbnb), or if listing has 0 reviews. If not, moves onto the next listing
    try: 
        # Click on 'reviews' link underneath listing name on top of the page to get all reviews on a pop-up
        reviews_button = driver.find_element(By.CSS_SELECTOR, "button[class='_koyt9le']")
        reviews_button.click()
        time.sleep(3)

        # You must scroll to the bottom of the pop-up window (which has infinite scroll) to get all reviews
        pop_up_window = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='_17itzz4']")))

        while True:
            # Get height of div, and height of div after scrolling until we've scrolled to the bottom
                partial_height = driver.execute_script('return arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;', pop_up_window)
            full_height = driver.execute_script('return arguments[0].scrollTop + arguments[0].offsetHeight;', pop_up_window)

            driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;', pop_up_window)
            time.sleep(0.5)

            if partial_height == full_height:
                break

        # Get reviews
        get_review_text = driver.find_elements(By.CSS_SELECTOR, "span[class='ll4r2nl dir dir-ltr']")
        all_reviews = [review.text for review in get_review_text if review.text != ""]

        # Get review corresponding dates
        get_review_dates = driver.find_elements(By.CSS_SELECTOR, "ol[class='_mac976w']")
        all_review_dates = [review.text for review in get_review_dates if review.text != ""]

        # Add each review with respective date to pandas dataframe
        for review, date in zip(all_reviews, all_review_dates):
            row = {'listing_id':listing_id, 'url':url, 'review_posted_date':date, 'review':review}
            all_reviews_df = all_reviews_df.append(row, ignore_index = True)

        all_reviews_df = all_reviews_df.drop_duplicates(ignore_index = True)
    
    except:
        continue
        
# Close browser window
driver.close()
    

In [221]:
# Check final df
all_reviews_df

,listing_id,url,review_posted_date,review
0,2595,https://www.airbnb.com/rooms/2595,November 2019,"Great location, convenient to everything. Very..."
1,2595,https://www.airbnb.com/rooms/2595,May 2019,Place was so cute and comfy! Host was great an...
2,2595,https://www.airbnb.com/rooms/2595,May 2019,10 / 10 would stay again
3,2595,https://www.airbnb.com/rooms/2595,January 2019,The apartment met expectations to how it was i...
4,2595,https://www.airbnb.com/rooms/2595,December 2018,Great space in a fun old building in NYC. Love...
...,...,...,...,...
17439,1918693,https://www.airbnb.com/rooms/1918693,February 2022,Lovely Brownstone in Brooklyn. Clean and spaci...
17440,1918693,https://www.airbnb.com/rooms/1918693,January 2022,We had a great stay at Lorelei & Alex’s place....
17441,1918693,https://www.airbnb.com/rooms/1918693,December 2021,This was a perfect spot for mine and my partne...
17442,1918693,https://www.airbnb.com/rooms/1918693,November 2021,A lovely spot in a lovely neighborhood. Great ...


In [226]:
# Count unique listings that were scraped
all_reviews_df.groupby('listing_id').nunique()

,url,review_posted_date,review
listing_id,,,
2595,1,34,48
5121,1,38,50
5178,1,100,231
5203,1,52,118
5803,1,105,206
...,...,...,...
1910270,1,6,7
1913540,1,2,2
1914185,1,20,22


In [209]:
# Change date from 'Month YYYY' to MM/YYYY format
all_reviews_df['review_posted_date'] = [(datetime.strptime(item, '%B %Y').date()).strftime('%m/%Y') for item in all_reviews_df['review_posted_date']]


In [222]:
# Print final reviews data to CSV file
all_reviews_df.to_csv('NYC_2021_airbnb_reviews_data.csv', index=False)